In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [42]:
df = pd.read_csv('ClosedStack_data_Virtual.csv', low_memory= False, encoding='latin-1')

In [43]:
df_new = df[[ 'Job ID','Hardware ID', 'Hardware Name', 'Solution ID', 'Ticket Site','Site Name', 'Job Created']]

In [126]:

df_new2019 = df_new.loc[(df_new['Job Created'] <= "2019-08-01 00:00:00.000000") & (df_new['Job Created'] >= "2019-01-01 00:00:00.000000")]
#take off minute seconds hour, deliniate based of space, year month day on dashes, change strings to integers
df_filtered = df_new2019.dropna(subset=['Job ID', 'Job Created', 'Solution ID'])
df_filtered.loc[:,'Job Created'] = pd.to_datetime(df_filtered['Job Created']).dt.strftime('%Y-%m-%d')
print (df_filtered.loc[:, 'Job Created'])


29790    2019-02-01
29796    2019-04-09
29797    2019-01-25
29798    2019-03-09
29799    2019-02-06
            ...    
86458    2019-01-02
86463    2019-01-02
86471    2019-01-02
86616    2019-01-02
86617    2019-01-02
Name: Job Created, Length: 19300, dtype: object


In [127]:
# One-Hot encode the 'Solution ID' column
onehot_encoder = OneHotEncoder()
solution_id_encoded = onehot_encoder.fit_transform(df_filtered[['Solution ID']]).toarray()

In [128]:

# One-Hot encode the categorical columns (in this case, 'Job ID')
categorical_cols = ['Job ID']
encoder = OneHotEncoder()
categorical_encoded = encoder.fit_transform(df_filtered[categorical_cols]).toarray()

In [129]:
onehot_encoder = OneHotEncoder()
jobcreated_encoded = onehot_encoder.fit_transform(df_filtered[['Job Created']]).toarray()

In [130]:
# Concatenate one-hot encoded 'Solution ID' and categorical columns
X_categorical = np.concatenate([categorical_encoded, solution_id_encoded], axis=1)

In [131]:
y = np.concatenate([jobcreated_encoded], axis=1)

In [132]:
# PCA for combined features
pca_categorical = PCA(n_components=50)
X_catagorical_pca = pca_categorical.fit_transform(X_categorical)

In [133]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_catagorical_pca, y, test_size=0.3, random_state=42)

X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

y_train = y_train.astype(np.float64)
y_test = y_test.astype(np.float64)

In [141]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [142]:

# Create DataLoader for training and testing data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [143]:
# Define custom neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # Input size is the number of features after PCA
        self.fc2 = nn.Linear(128, 3456)  # Output size is 1 for regression task

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [144]:
# Initialize the model, loss function, and optimizer
model = NeuralNetwork(input_size=X_catagorical_pca.shape[1])
criterion = nn.MSELoss()  # Mean Squared Error loss for regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [145]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        

        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

c:\Users\Alaina\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64, 178])) that is different to the input size (torch.Size([64, 3456])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (3456) must match the size of tensor b (178) at non-singleton dimension 1

In [139]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    mse = criterion(test_outputs, y_test_tensor)
    print("Mean Squared Error on Test Data:", mse.item())

c:\Users\Alaina\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([5790, 1, 178])) that is different to the input size (torch.Size([5790, 3456])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (3456) must match the size of tensor b (178) at non-singleton dimension 2

In [11]:
df['Solution ID'].unique()

array(['03145e08-3c9f-4514-9acc-02a1657fee80',
       '71e4908b-9e81-4641-929c-08dd9876a69b',
       '9aa697ea-ca66-4064-b669-096c647d26a4',
       '1ac664c0-34dd-4f8d-90ae-262319c76f5d',
       '897c2511-c6ef-4552-ba23-8744dd6ea439',
       'c01f7ab8-d51a-4234-9826-4e814f592368',
       'a4c66551-9b21-454a-8929-61f8fe2fc37e',
       'db503ea2-e70d-47bd-9a11-f76a5b30c0b3',
       'eaff2e4c-a5ba-4547-a5bd-6946885ebe3e',
       '19ad77e9-b248-4239-90d7-9a463496834e',
       '2f099dbb-7ab6-42de-9bff-8ab0fd4632e8',
       'af088fd4-c01c-44c6-b985-4db915230b01',
       '0b7f55e7-f2ed-4499-9bcb-a249825e5502',
       '2d020295-2a92-4789-9963-e55864097293',
       '86c7197c-ab92-4ba6-9ef2-93f44d2e66d5',
       '35f9c687-de0f-4a77-8e21-eac3ed623fcb',
       '56acc9f9-f460-4c27-b904-e54b1d6c1703',
       '25888a34-c2d1-4bdb-9926-8b13822820cd', nan,
       'a669d4f4-fef6-4a96-a0b8-12cb5d61b8e6',
       'bee59aba-5835-491b-93d5-26c73f882900',
       '91134aac-fd3b-4acb-b3f5-22ff63e6fe43',
       '